# Step 0 Set up `kgtk`
Check `kgtk` GitHub with branch `dev`;

# Step 1 User Query

Example description: find industry of companies;

In [1]:
import os

### Define alias and variables

In [3]:
# Parameters

# Folder where database files store
data_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"

# Folder on local machine where to create the output and temporary folders
output_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/output/"

# Location of the cache database for kypher
cache_path = "/nas/home/bohuizha/KG/hunger-for-knowledge/kypher"
# Whether to delete the cache database
delete_database = False

# The names of files in the KGTK Wikidata distirbution that we will use in this notebook.
data_file_names = {
    "claims": "claims.tsv",
    "wiki_info": "wikidata_infobox.tsv",
    "p31": "P31.tsv",
    "p279star": "P279star.tsv",
    "labels": "labels.en.tsv"
}

# We will define environment variables to hold the full paths to the files as we will use them in the shell commands
kgtk_environment_variables = []

os.environ['DATABASE'] = data_path
kgtk_environment_variables.append('DATABASE')

for key, value in data_file_names.items():
    variable = key.upper()
    os.environ[variable] = data_path + value
    kgtk_environment_variables.append(variable)

output_file_names = {
    "results": "industry_of_company.tsv",
    "new_results": "new_industry_of_company.tsv",
    "entity": "company.tsv",
    "query_file": "company_wo_industry.tsv",
    "direct_infer": "direct_infer.tsv",
    "indirect_infer": "indirect_infer.tsv",
    "infers": "infers.tsv",
    "numbers": "numbers.tsv",
    "strings": "strings.tsv",
    "empty_strings": "empty_strings.tsv",
    "pure_empty": "pure_empty.tsv",
    "non_empty": "non_empty_strings.tsv",
    "structured_literals": "structured_literals.tsv",
    "nodes": "nodes.tsv",
    "qnodes": "qnodes.tsv",
    "correct_temp_1": "industry.type-constraints.instanceOf.correct_temp_1.tsv",
    "correct_temp_2": "industry.type-constraints.instanceOf.correct_temp_2.tsv",
    "incorrect_temp": "industry.type-constraints.instanceOf.incorrect_temp.tsv",
    "correct": "industry.type-constraints.instanceOf.correct.tsv",
    "incorrect": "industry.type-constraints.instanceOf.incorrect_temp.tsv"
}

os.environ['OUTPUT'] = output_path
kgtk_environment_variables.append('OUTPUT')

for key, value in output_file_names.items():
    variable = key.upper()
    os.environ[variable] = output_path + value
    kgtk_environment_variables.append(variable)

# KGTK creates a SQLite database to index the knowledge graph.
if cache_path:
    os.environ['STORE'] = "{}/wikidata.sqlite3.db".format(cache_path)
else:
    os.environ['STORE'] = "{}/{}/wikidata.sqlite3.db".format(output_path, temp_folder)
kgtk_environment_variables.append('STORE')
    
# Envronment variables with shortcuts to the commands we use often
# os.environ['kgtk'] = kgtk
# Use for debugging, but careful as it causes import to dataframes to break
# os.environ['kypher'] = "time kgtk --debug query --graph-cache " + os.environ['STORE']
os.environ['kypher'] = "kgtk query --graph-cache " + os.environ['STORE']
# kgtk_environment_variables.append('kgtk')
kgtk_environment_variables.append('kypher')
    
# kgtk_environment_variables.sort()
for variable in kgtk_environment_variables:
    print("{}: \"{}\"".format(variable, os.environ[variable]))

DATABASE: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/"
CLAIMS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/claims.tsv"
WIKI_INFO: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/wikidata_infobox.tsv"
P31: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P31.tsv"
P279STAR: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/P279star.tsv"
LABELS: "/nas/home/bohuizha/KG/hunger-for-knowledge/data/labels.en.tsv"
OUTPUT: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/"
RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/industry_of_company.tsv"
NEW_RESULTS: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/new_industry_of_company.tsv"
ENTITY: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/company.tsv"
QUERY_FILE: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/company_wo_industry.tsv"
DIRECT_INFER: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/direct_infer.tsv"
INDIRECT_INFER: "/nas/home/bohuizha/KG/hunger-for-knowledge/output/indirect_infer.tsv"
INFERS: "/n

# Step 2: Wikidata Results

In [1]:
# SPARQL query: 
# SELECT DISTINCT ?companyLabel ?industryLabel 
# WHERE
# { 
#   ?company wdt:P31/wdt:P279*  wd:Q783794 ;
#            wdt:P452  ?industry . 
#   SERVICE wikibase:label { bd:serviceParam wikibase:language "en". } 
# } 
# where `P106` means "occupation", `Q82955` means "politician", `P26` means "spouse";

In [25]:
!kgtk query -i $CLAIMS -i $P31 -i $P279STAR -i $LABELS \
            --match 'P31: (company)-[]->(class), P279star: (class)-[]->(:Q783794), claims: (company)-[p:P452]->(industry), labels: (industry)-[]->(industry_label)' \
            --return 'company as node1, "P452" as label, industry_label as node2' \
            --limit 10

node1	label	node2
Q1000076	P452	'manufacturing'@en
Q1000428	P452	'software industry'@en
Q1000752	P452	'public transport'@en
Q1001167	P452	'automotive industry'@en
Q1001321	P452	'automotive industry'@en
Q1001330	P452	'publisher'@en
Q1001380	P452	'video game industry'@en
Q1001516	P452	'drugstore'@en
Q1001516	P452	'retail'@en
Q1001883	P452	'automotive industry'@en


In [26]:
!kgtk query -i $CLAIMS -i $P31 -i $P279STAR \
            --match 'P31: (company)-[]->(class), P279star: (class)-[]->(:Q783794), claims: (company)-[p:P452]->(industry)' \
            --return 'company as node1, "P452" as label, industry as node2' \
            -o $RESULTS

Check head of the results:

In [27]:
!head $RESULTS | column -ts $'\t'

node1     label  node2
Q1000076  P452   Q187939
Q1000428  P452   Q880371
Q1000752  P452   Q178512
Q1001167  P452   Q190117
Q1001321  P452   Q190117
Q1001330  P452   Q2085381
Q1001380  P452   Q941594
Q1001516  P452   Q1260046
Q1001516  P452   Q126793


### Count known results in Wikidata database:

Count entity-value pairs / **rows** (result should -1 which is the header):

In [8]:
!wc -l $RESULTS

19282 /nas/home/bohuizha/KG/hunger-for-knowledge/output/industry_of_company.tsv


Count how many **unique entities** have property in Wikidata:

In [9]:
!kgtk query -i $RESULTS \
            --match '(p)-[]->()' \
            --return 'count(distinct p) as N'

N
16056


### Find unknow results in Wikidata database:
- Find all entities

In [11]:
!kgtk query -i $P31 -i $P279STAR \
            --match 'P31: (entity)-[]->(class), P279star: (class)-[]->(:Q783794)' \
            --return 'entity as node1, "P31" as label, "Q783794" as node2' \
            -o $ENTITY

- Eliminate entities who have property / properties

In [12]:
!kgtk ifnotexists -i $ENTITY \
                  --filter-on $RESULTS \
                  --input-keys node1 \
                  --filter-keys node1 \
                  -o $QUERY_FILE

### Count unknown results in Wikidata database:

In [13]:
!kgtk query -i $QUERY_FILE \
            --match '(p)-[]->()' \
            --return 'count(distinct p) as N'

N
265805


# Step 3 Selection of Additional KG(s)
- DBpedia
- Getty
- Freebase
- Company KGs

# Step 4 Schema Alignment
## Entity resolution

Use query results from Wikidata database to infer properties in Wikidata infobox and return the most frequent property.

In [33]:
!kgtk query -i $RESULTS -i $WIKI_INFO \
            --match 'c: (entity)-[]->(v), w: (entity)-[p]->(v)' \
            --return 'entity, p.label, v as node2' \
            -o $DIRECT_INFER

Infer from structured literals (since there is no string structured value in Wikidata infobox).

In [42]:
!kgtk query -i $RESULTS -i $WIKI_INFO -i $LABELS \
            --match 'c: (entity)-[]->(v1), l: (v1)-[]->(v1_label), w: (entity)-[p]->(s)-[sv]->(v2)' \
            --where 'sv.label = "dbpedia:structured_value" AND kgtk_lqstring_text(v1_label) = kgtk_lqstring_text(v2)' \
            --return 'entity, p.label, v2 as node2' \
            -o $INDIRECT_INFER

In [44]:
!kgtk cat -i $DIRECT_INFER $INDIRECT_INFER -o $INFERS

## Property mapping

In [48]:
!kgtk query -i $INFERS \
            --match '(q)-[p]->(v)' \
            --return 'p.label, count(v) as N' \
            --order-by 'N desc' \
            --limit 1

label	N
property:industry	4608


# Step 5 Results from other KG(s)

For those entities don't have property value, query in Wikidata infobox:

In [49]:
!kgtk query -i $QUERY_FILE -i $WIKI_INFO \
            --match 'p: (entity)-[]->(), w: (entity)-[property]->(value)' \
            --where 'property.label = "property:industry"' \
            --return 'entity, property.label, value' \
            -o $NEW_RESULTS

- Count rows of new findings:

In [50]:
new_results_line = !wc -l < $NEW_RESULTS
new_results_line = int(new_results_line[0]) - 1
# new_results_line

4264

- Count unique politicians of new findings:

In [51]:
new_results_distinct = !kgtk query \
    -i $NEW_RESULTS \
    --match 'n: (p)-[]->()' \
    --return 'count(distinct p) as N'
new_results_distinct = int(new_results_distinct[1])
# new_results_distinct

3499

# Step 6 Datatype Filtering

### 1. Filter Structured literals:

In [35]:
!kgtk query -i $NEW_RESULTS -i $WIKI_INFO \
            --match 'n: (q)-[p]->(s), w: (s)-[sv]->(v)' \
            --where 'NOT kgtk_lqstring(s) AND NOT kgtk_number(s) AND sv.label = "dbpedia:structured_value"' \
            --return 'q, p.label, s' \
            -o $STRUCTURED_LITERALS

- Check if data type is useful:

In [36]:
!head $STRUCTURED_LITERALS | column -ts $'\t'

node1      label              node2
Q25348727  property:industry  nodemxZbyK2VRrGoaxfdLmyLxw-4089958


### 2. Filter Qnodes

In [37]:
!kgtk query -i $NEW_RESULTS \
            --match 'n:()-[]->(q)' \
            --where 'NOT kgtk_lqstring(q) AND NOT kgtk_number(q)' \
            -o $NODES

!kgtk ifnotexists -i $NODES \
                  --filter-on $STRUCTURED_LITERALS \
                  -o $QNODES

- Check if the data type is useful:

In [40]:
!head $QNODES | column -ts $'\t'

node1      label              node2
Q1023161   property:industry  Q269415
Q1024380   property:industry  Q5501371
Q1044059   property:industry  Q815825
Q1044059   property:industry  Q49389
Q1060363   property:industry  Q899383
Q1060363   property:industry  Q831882
Q1060363   property:industry  Q1020768
Q1073564   property:industry  Q778575
Q10831597  property:industry  Q778575


In [41]:
!wc -l < $QNODES

1871


# Step 7 Quality Checking

Problem: no `nodeProp.id`;

In [42]:
!kgtk --debug query -i $QNODES $P31 $P279STAR \
--match 'n: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(nodex), P279star: (nodex)-[]->(par)' \
--where 'par in ["Q8148", "Q268592", "Q8187769", "Q3958441", "Q121359"] ' \
--return 'node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
-o $CORRECT_TEMP_1

[2021-08-26 05:32:48 sqlstore]: DROP graph data table graph_65 from /nas/home/bohuizha/KG/hunger-for-knowledge/output/qnodes.tsv
[2021-08-26 05:32:48 sqlstore]: IMPORT graph directly into table graph_65 from /nas/home/bohuizha/KG/hunger-for-knowledge/output/qnodes.tsv ...
[2021-08-26 05:32:48 query]: SQL Translation:
---------------------------------------------
  SELECT graph_65_c1."node1" "_aLias.node1", graph_65_c1."label" "_aLias.label", graph_65_c1."node2" "_aLias.node2"
     FROM graph_56 AS graph_56_c2
     INNER JOIN graph_57 AS graph_57_c3, graph_65 AS graph_65_c1
     ON graph_56_c2."node2" = graph_57_c3."node1"
        AND graph_65_c1."node2" = graph_56_c2."node1"
        AND (graph_57_c3."node2" IN (?, ?, ?, ?, ?))
  PARAS: ['Q8148', 'Q268592', 'Q8187769', 'Q3958441', 'Q121359']
---------------------------------------------
[2021-08-26 05:32:48 sqlstore]: CREATE INDEX on table graph_65 column node2 ...
[2021-08-26 05:32:48 sqlstore]: ANALYZE INDEX on table graph_65 column n

In [43]:
!head $CORRECT_TEMP_1 | column -ts $'\t'

node1      label              node2
Q1044059   property:industry  Q49389
Q11617252  property:industry  Q8274
Q11813614  property:industry  Q418
Q11998559  property:industry  Q75
Q11998559  property:industry  Q75
Q12315533  property:industry  Q3972943
Q12315533  property:industry  Q3972943
Q1236521   property:industry  Q49389
Q12430250  property:industry  Q507443


In [44]:
!kgtk --debug ifnotexists -i $QNODES \
--filter-on $CORRECT_TEMP_1 \
-o $INCORRECT_TEMP

In [45]:
!head $INCORRECT_TEMP | column -ts $'\t'

node1      label              node2
Q1023161   property:industry  Q269415
Q1024380   property:industry  Q5501371
Q1044059   property:industry  Q815825
Q1060363   property:industry  Q899383
Q1060363   property:industry  Q831882
Q1060363   property:industry  Q1020768
Q1073564   property:industry  Q778575
Q10831597  property:industry  Q778575
Q1085796   property:industry  Q291


In [46]:
!kgtk --debug query -i $INCORRECT_TEMP $P31 \
--match 'i: (node1)-[nodeProp]->(node2), P31: (node2)-[]->(par)' \
--where 'par in ["Q8148", "Q268592", "Q8187769", "Q3958441", "Q121359"] ' \
--return 'node1 as `node1`, nodeProp.label as `label`, node2 as `node2`' \
-o $CORRECT_TEMP_2

[2021-08-26 05:33:20 sqlstore]: DROP graph data table graph_74 from /nas/home/bohuizha/KG/hunger-for-knowledge/output/industry.type-constraints.instanceOf.incorrect_temp.tsv
[2021-08-26 05:33:20 sqlstore]: IMPORT graph directly into table graph_74 from /nas/home/bohuizha/KG/hunger-for-knowledge/output/industry.type-constraints.instanceOf.incorrect_temp.tsv ...
[2021-08-26 05:33:20 query]: SQL Translation:
---------------------------------------------
  SELECT graph_74_c1."node1" "_aLias.node1", graph_74_c1."label" "_aLias.label", graph_74_c1."node2" "_aLias.node2"
     FROM graph_56 AS graph_56_c2
     INNER JOIN graph_74 AS graph_74_c1
     ON graph_74_c1."node2" = graph_56_c2."node1"
        AND (graph_56_c2."node2" IN (?, ?, ?, ?, ?))
  PARAS: ['Q8148', 'Q268592', 'Q8187769', 'Q3958441', 'Q121359']
---------------------------------------------
[2021-08-26 05:33:20 sqlstore]: CREATE INDEX on table graph_74 column node2 ...
[2021-08-26 05:33:20 sqlstore]: ANALYZE INDEX on table graph_

In [47]:
!head $CORRECT_TEMP_2 | column -ts $'\t'

node1  label  node2


In [14]:
!kgtk --debug ifnotexists -i $INCORRECT_TEMP \
--filter-on $CORRECT_TEMP_2 \
-o $INCORRECT

Traceback (most recent call last):
  File "/nas/home/bohuizha/.conda/envs/kgtk-env/lib/python3.7/site-packages/kgtk-0.7.1-py3.7.egg/kgtk/cli/ifnotexists.py", line 257, in run
    ie.process()
  File "/nas/home/bohuizha/.conda/envs/kgtk-env/lib/python3.7/site-packages/kgtk-0.7.1-py3.7.egg/kgtk/iff/kgtkifexists.py", line 943, in process
    join_shuffle_list=join_shuffle_list)
  File "/nas/home/bohuizha/.conda/envs/kgtk-env/lib/python3.7/site-packages/kgtk-0.7.1-py3.7.egg/kgtk/iff/kgtkifexists.py", line 222, in process_cacheing_filter
    input_key = self.build_key(row, input_key_columns)
  File "/nas/home/bohuizha/.conda/envs/kgtk-env/lib/python3.7/site-packages/kgtk-0.7.1-py3.7.egg/kgtk/iff/kgtkifexists.py", line 155, in build_key
    key += row[idx]
IndexError: list index out of range

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/nas/home/bohuizha/.conda/envs/kgtk-env/lib/python3.7/site-packages/kgtk-0.7.1-py3.7.egg/k

It seems like if `$CORRECT_TEMP_2` is empty, then there is no need to filter again.

In [48]:
!kgtk --debug cat -i $CORRECT_TEMP_1 $CORRECT_TEMP_2 \
-o $CORRECT

In [49]:
!head $CORRECT | column -ts $'\t'

node1      label              node2
Q1044059   property:industry  Q49389
Q11617252  property:industry  Q8274
Q11813614  property:industry  Q418
Q11998559  property:industry  Q75
Q11998559  property:industry  Q75
Q12315533  property:industry  Q3972943
Q12315533  property:industry  Q3972943
Q1236521   property:industry  Q49389
Q12430250  property:industry  Q507443


In [50]:
!wc -l < $CORRECT

425


In [51]:
!wc -l < $INCORRECT_TEMP

1568


In [52]:
!wc -l < $QNODES

1871


It is weird that 425 + 1568 - 1 = 1992 > 1871. (+121)

In [53]:
!kgtk --debug ifexists -i $CORRECT \
--filter-on $INCORRECT_TEMP

node1	label	node2
